<a href="https://colab.research.google.com/github/vchiang001/sexual_diversity_dissertation/blob/main/Curating_camera_trap_dataset_for_future_memory_replay_fine_tuning_of_DeepLabCut_quadruped_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Which images fulfill criteria

In [ ]:
###import packages
import json
import shutil
import os

In [ ]:
json_path = '' #<location of json file>
mega_thresh = 0.01 #you can set it at a number between 0.0 – 1.0 (the higher the more confident)

In [ ]:
with open(json_path, 'r') as f:
    detection_results = json.load(f)

mre2anim = []
for img_data in detection_results['images']:
    if len(img_data['detections']) >= 2:
        anim_detect = 0
        print(img_data['file'])
        for detect in img_data['detections']:
            if detect['category'] == '1' and detect['conf'] >= mega_thresh:
                anim_detect += 1
        if anim_detect >= 2:
            mre2anim.append(img_data)#add the img name to a list or another file type
print("number of image with more than 2 animals:", len(mre2anim)) #list how many images fulfill this

###from the list, find how many images with potentially interacting animals
# function to check for bounding box overlaps
def overlap(bbox1, bbox2):
    #rectangle parameter of bbox1
    width1 = bbox1[2]
    height1 = bbox1[3]

    #rectangle parameter of bbox2
    width2 = bbox2[2]
    height2 = bbox2[3]

    #left btm of bbox1
    if bbox2[0] <= bbox1[0] <= (bbox2[0] + width2) and \
        bbox2[1] <= bbox1[1] <= (bbox2[1] + height2):
        return True

    #left top of bbox1
    if bbox2[0] <= bbox1[0] <= (bbox2[0] + width2) and \
        bbox2[1] <= (bbox1[1]+height1) <= (bbox2[1] + height2):
        return True

    #right btm of bbox1
    if bbox2[0] <= (bbox1[0]+width1) <= (bbox2[0] + width2) and \
        bbox2[1] <= bbox1[1] <= (bbox2[1] + height2):
        return True

    #right top of bbox1
    if bbox2[0] <= (bbox1[0]+width1) <= (bbox2[0] + width2) and \
        bbox2[1] <= (bbox1[1]+height1) <= (bbox2[1] + height2):
        return True

    return False

#check if bbox interacts
bbox_int = []
for img_data in mre2anim:
    for detect in img_data['detections']:
        if detect['conf'] >= mega_thresh:
            bbox1 = detect['bbox']
            for detect2 in img_data['detections']:
                if detect2 != detect:
                    if detect2['conf'] >= mega_thresh:
                        bbox2 = detect2['bbox']
                        if overlap(bbox1, bbox2) == True:
                            print(img_data['file'])
                            bbox_int.append(img_data['file']) #unlike mre2anim, you just want the file name

bbox_int_uniq = set(bbox_int) #get only the unique records in the list
bbox_int_uniq_list = list(bbox_int_uniq)
print("number of image with interacting animals:", len(bbox_int_uniq_list)) #list how many images fulfill this

###Downloading subset of camera trap images fulfilling criteria

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!curl https://sdk.cloud.google.com | bash

In [ ]:
!gcloud init

In [ ]:
url_base = 'gs://public-datasets-lila/wellington-unzipped/images/' #change url for camera trap datasets of interest
output_folder = "" #<location of where to save files>
bbox_int_uniq_list_url = []
for img in bbox_int_uniq_list:
  url = url_base + img
  bbox_int_uniq_list_url.append(url)

In [ ]:
# Loop through the URLs and download files
for url in bbox_int_uniq_list_url:
    !gsutil -m cp "$url" "$output_folder"

###Running MegaDetector on subset of images

In [ ]:
images_dir = '' #directory of your subsets
output_file_path = ''# Choose a location for the output JSON file

In [ ]:
#Detection script on batch data
!python /content/CameraTraps/detection/run_detector_batch.py md_v5b.0.0.pt "$images_dir" "$output_file_path" --recursive --output_relative_filenames --quiet

In [ ]:
#Visualisation
visualization_dir = '/content/visualized_images'
!python /content/CameraTraps/visualization/visualize_detector_output.py "$output_file_path" "$visualization_dir" --confidence 0.2 --images_dir "$images_dir"